In [115]:
import xarray as xr
import rioxarray as rio
import glob
from pathlib import Path
import natsort
import subprocess
import numpy as np

In [111]:
files = natsort.natsorted(glob.glob('windmapper_config/*.tif'))

# the shortest name is the the base DEM. Remove this basename from the variable name used in the dataset
basename = Path(min(files, key=len)).stem

variables = []
dem = None
for tif_file in files:
    # Use the file name (without extension) as the variable name
    variable_name = Path(tif_file).stem

    if variable_name == basename:

        # The WM area is smaller than the input DEM due to the strict alignment requirements
        # cut down the input DEM to match the output WM extent
        variable_name = variable_name.replace(basename,'DEM')
        shp = Path(tif_file).parent / 'shp' / 'user_bbox.shp'
        new_dem = 'DEM-WM-clip.tif'

        command = [
            "gdalwarp",  # Replace with actual path to gdalwarp
            "-cutline", shp,
            "-crop_to_cutline",
            "-dstalpha",
            "-overwrite",
            'FABDEM-clip.tif',
            new_dem
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        dem = xr.open_mfdataset(new_dem).isel(band=0).drop_vars('band')
        dem = dem.rename({'band_data': 'DEM','x':'lon','y':'lat'})
        continue
        
    else:
        variable_name = variable_name.replace(basename,'dir')
    
    ds = xr.open_mfdataset(tif_file).isel(band=0).drop_vars('band')
    ds = ds.rename({'band_data': variable_name,'x':'lon','y':'lat'})
    variables.append(ds)

# Combine all variables into a single xarray.Dataset
# Enforce them to all /exactly/ match the same x,y and compensate for any tiny floating point differences in the lat lon coords 
ds = xr.merge( xr.align(*variables, join="override"), compat='no_conflicts', join='exact')


In [113]:
ds

<xarray.Dataset> Size: 5MB
Dimensions:              (lon: 267, lat: 195)
Coordinates:
  * lon                  (lon) float64 2kB -115.3 -115.3 ... -115.2 -115.2
  * lat                  (lat) float64 2kB 50.9 50.9 50.89 ... 50.78 50.78 50.78
    spatial_ref          int64 8B 0
Data variables: (12/24)
    dir_0_U              (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_0_V              (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_0_spd_up_1000    (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_45_U             (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_45_V             (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_45_spd_up_1000   (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    ...                   ...
    dir_270_U            (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_270_V            (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_270_spd_up_1000  (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_315_U            (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_315_V            (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>
    dir_315_spd_up_1000  (lat, lon) float32 208kB dask.array<chunksize=(7, 267), meta=np.ndarray>

In [114]:
# Create a new dataset with the same coordinates as the existing dataset
wind = xr.Dataset(coords=ds.coords)

# Add a new variable to the new dataset (optional)
wind["speed"] = (("lon", "lat"), np.zeros((267,195))+10)

NameError: name 'np' is not defined